# Assignment 3: Impact of low-skill immigration on the skill premium
#### By: Augusto Ospital. May 17, 2023, Updated version: November 5th, 2023 (Mallika Chandra)

Data: 1990 and 2000 Census, and CONSPUMA as a spatial unit.

__Step I__. Construct native labor-market outcomes by CONSPUMA $c$, year $y$ (1990 and 2000), and education $e$ ($e = H$ for ≥ 12 years schooling vs. $L$ for < 12 years)

__Step II__. Construct working-age population with ($e = H$) and without ($e = L$) more than 12 years of schooling by $cy$, $N^e_{c,y}$

__Step III__. Measure the change in the share with ≤ 12 years of schooling:
$$ x_c = \frac{N^L_{c,2000}}{N^L_{c,2000}+N^H_{c,2000}} - \frac{N^L_{c,1990}}{N^L_{c,1990}+N^H_{c,1990}}$$

__Step IV__. Construct instrument:
$$ z_c 
= \frac{(I^{Mex}_{c,1990}/I^{Mex}_{1990})(I^{Mex}_{c,2000}-I^{Mex}_{c,1990})}{N^{L}_{c,1990}+N^{H}_{c,1990}} 
= \frac{I^{Mex}_{c,1990}}{N^{L}_{c,1990}+N^{H}_{c,1990}} \frac{I^{Mex}_{c,2000}-I^{Mex}_{c,1990}}{I^{Mex}_{1990}}
$$
where
- $I^{Mex}_{c,y} = $ mexican population of $c$ in year $y$
- $I^{Mex}_{y} = $ total mexican population in year $y$

__Step V__. Using 2SLS, project changes in CONSPUMA relative outcomes for higher vs. lower education on $x_c$, instrumenting with $z_c$

## Code Preliminaries

In [1]:
from pathlib import Path
import pandas as pd
from econtools import group_id
import numpy as np

In [2]:
mainp = Path('/Users/augusto/Dropbox/UCLA Classes/Teaching/econ424_S23/assig3')

In [3]:
# For regressions
import statsmodels.api as sm
from stargazer.stargazer import Stargazer #nice tables with statsmodels
from linearmodels.iv import IV2SLS, compare #2sls with clustered SEs
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['font.size'] = 11
plt.rcParams['font.family'] = 'serif'

## Prepare the Data

#### Load the data from IPUMS

In [4]:
# Take a look at column definitions:
pd.read_stata(mainp / 'files_provided'/ 'usa_00122.dta', iterator=True).variable_labels()

{'year': 'census year',
 'statefip': 'state (fips code)',
 'conspuma': 'consistent puma, 1980-1990-2000',
 'gq': 'group quarters status',
 'perwt': 'person weight',
 'sex': 'sex',
 'age': 'age',
 'race': 'race [general version]',
 'bpl': 'birthplace [general version]',
 'educ': 'educational attainment [general version]',
 'empstat': 'employment status [general version]',
 'ind1990': 'industry, 1990 basis',
 'wkswork1': 'weeks worked last year',
 'uhrswork': 'usual hours worked per week',
 'incwage': 'wage and salary income'}

In [5]:
df = pd.read_stata(mainp/'files_provided'/'usa_00122.dta', convert_categoricals=False)

# Choosing a baseline sample
df = df[(df.age>=18) & (df.age<=65) & (df.gq<=2)].copy() # Keep those aged 18-65 and not in group quarters


In [7]:
# Summary statistics
def human_format(num):
    magnitude = 0
    while abs(num) >= 1000:
        magnitude += 1
        num /= 1000.0
    # add more suffixes if you need them
    return '%.2f%s' % (num, ['', 'K', 'Million', 'Trillion', 'G', 'P'][magnitude])

# df.describe().applymap('{:,.0f}'.format).T
df.describe().applymap(human_format).T

,count,mean,std,min,25%,50%,75%,max
year,15.94Million,2.00K,4.99,1.99K,1.99K,2.00K,2.00K,2.00K
statefip,15.94Million,27.68,15.77,1.00,13.00,27.00,41.00,56.00
conspuma,15.94Million,250.41,163.87,1.00,88.00,257.00,390.00,543.00
gq,15.94Million,1.00,0.05,1.00,1.00,1.00,1.00,2.00
perwt,15.94Million,20.19,9.91,0.00,14.00,20.00,26.00,345.00
sex,15.94Million,1.51,0.50,1.00,1.00,2.00,2.00,2.00
age,15.94Million,39.60,12.87,18.00,29.00,39.00,50.00,65.00
race,15.94Million,1.62,1.62,1.00,1.00,1.00,1.00,9.00
bpl,15.94Million,69.31,120.48,1.00,19.00,36.00,48.00,900.00
educ,15.94Million,6.81,2.33,0.00,6.00,6.00,8.00,11.00


In [8]:
df['is_1990'] = np.where(df.year==1990, 1, 0)
df['is_2000'] = np.where(df.year==2000, 1, 0)

# Demographics:
df['is_native'] = np.where(df.bpl<=120, 1, 0)
df['is_foreign'] = np.where((df.bpl>120)&(df.bpl<900), 1, 0)
df['is_female'] = np.where(df.sex==2, 1, 0)
df['is_mex'] = np.where(df.bpl==200, 1, 0) #mexican dummy

# Education:
df['is_col'] = np.where(df.educ>=10, 1, 0)
df['is_hs'] = np.where(df.educ>=6, 1, 0)
df['is_low'] = np.where(df.educ<6, 1, 0) #low education dummy (less than HS)

# Employment status:
df['is_emp'] = np.where(df.empstat==1, 1, 0)
df['is_unemp'] = np.where(df.empstat==2, 1, 0)
df['is_nilf'] = np.where(df.empstat==3, 1, 0)
# Manufacturing employment:
df['is_manuf'] = np.where((df.is_emp==1) & (df.ind1990>=100) & (df.ind1990<400), 1, 0)
df['is_nonmanuf'] = np.where((df.is_emp==1) & ((df.ind1990<100) | (df.ind1990>=400)), 1, 0)

In [9]:
# For precision in sums:
df = df.astype(np.float64).copy()

In [10]:
# for numerator of instrument
df['mex_y'] = df.is_mex*df.perwt
df['mex_y'] = df.groupby('year')['mex_y'].transform(sum) #number of Mexicans per year

In [11]:
# c-y controls and weight

df['mex_cy'] = df.is_mex * df.perwt
df['pop_cy'] = df.perwt
df['manuf_cy']= df.is_manuf * df.is_emp * df.perwt
df['female_cy'] = df.is_female * df.is_emp * df.perwt
df['emp_cy'] = df.is_emp * df.perwt
df['col_cy'] = df.is_col * df.perwt
df['hs_cy'] = df.is_hs * df.perwt
df['fborn_cy'] = df.is_foreign * df.perwt
df['nborn_cy'] = df.is_native * df.perwt
for col in [c for c in df.columns if '_cy' in c]:
    df[col] = df.groupby(['year','conspuma'])[col].transform(sum)

df['manuf_share_cy'] = df['manuf_cy']/df['emp_cy'] # manufacturing share of employed
df['female_share_cy'] = df['female_cy'] / df['emp_cy'] # female share of employed
df['col_share_cy'] = df['col_cy'] / df['pop_cy'] # college share of population
df['hs_share_cy'] = df['hs_cy'] / df['pop_cy'] # high school share of population
df['lnpop_cy']= np.log(df['pop_cy']) # log of population (in age range)
df.loc[df['lnpop_cy']==-np.inf, 'lnpop_cy'] = np.nan
df['fborn_share_cy'] = df['fborn_cy']/(df['fborn_cy']+df['nborn_cy']) # foreign-born share of employed
df['mex_share_cy'] = df['mex_cy']/df['pop_cy'] # Mexican share of population    

cy_cols = ['mex_cy','manuf_share_cy','female_share_cy','col_share_cy','hs_share_cy','lnpop_cy','fborn_share_cy','mex_share_cy']

In [12]:
# c-e-y outcomes for natives by conspuma, education, year
df['hours'] = df['uhrswork'] * df['wkswork1']
df['pop_cey'] = df.groupby(['conspuma','is_low','year'])['perwt'].transform(sum)

df = df[df['bpl']<100].copy() # excluding US OUTLYING AREAS/TERRITORIES

df['nilf_cey'] = df['is_nilf'] * df['perwt']
df['unemp_cey'] = df['is_unemp'] * df['perwt']
df['emp_cey'] = df['is_emp'] * df['perwt']
df['inc_cey'] = df['incwage'] * df['perwt']
df['hours_cey'] = df['hours'] * df['perwt']
for col in ['nilf_cey','unemp_cey','emp_cey','inc_cey','hours_cey']:
   df[col] = df.groupby(['year','conspuma','is_low'])[col].transform(sum)

df['nilf_rate_cey'] = df['nilf_cey'] / (df['nilf_cey']+df['unemp_cey']+df['emp_cey'])
df['unemp_rate_cey'] = df['unemp_cey'] / (df['unemp_cey']+df['emp_cey'])
df['ln_wage_cey'] = np.log(df['inc_cey']/df['hours_cey'])
df.loc[df['ln_wage_cey']==-np.inf, 'ln_wage_cey'] = np.nan

cey_cols = ['pop_cey','nilf_rate_cey','unemp_rate_cey','ln_wage_cey']

In [13]:
#keep one observation per conspuma x education x year
to_keep = ['conspuma','statefip','is_low','year','mex_y', *cy_cols, *cey_cols]
df = df[to_keep].copy()
df.drop_duplicates(inplace=True)

In [14]:
# reshape to one obserevation per conspuma x education
cols = df.columns.to_list() #save the column names
df = df.reset_index().pivot_table(index=['conspuma','statefip','is_low'], columns='year')

In [15]:
# get one observation per conspuma

df['dnilf_rate_ce'] = df['nilf_rate_cey',2000] - df['nilf_rate_cey',1990]
df['dunemp_rate_ce'] = df['unemp_rate_cey',2000] - df['unemp_rate_cey',1990]
df['dln_wage_ce'] = df['ln_wage_cey',2000] - df['ln_wage_cey',1990]

df.drop(['nilf_rate_cey','unemp_rate_cey','ln_wage_cey'], axis=1, inplace=True)

df = df.pivot_table(index=['conspuma','statefip'], columns=['is_low'])

In [16]:
# create double differenced outcome variables (our Ys)

df['Dnilf_rate_c'] = df['dnilf_rate_ce','',0] - df['dnilf_rate_ce','',1]
df['Dunemp_rate_c'] = df['dunemp_rate_ce','',0] - df['dunemp_rate_ce','',1]
df['Dln_wage_c'] = df['dln_wage_ce','',0] - df['dln_wage_ce','',1]

df.drop(['dnilf_rate_ce','dunemp_rate_ce','dln_wage_ce'], axis=1, inplace=True)

In [17]:
# create our X and regression weight
df['weight'] = df['pop_cey',1990,0]+df['pop_cey',1990,1]
df['x'] = df['pop_cey',2000,1]/(df['pop_cey',2000,0]+df['pop_cey',2000,1]) \
          - df['pop_cey',1990,1]/df['weight']

df.drop('pop_cey', axis=1, inplace=True)

In [18]:
# create our instrument
df['z'] = (1/df['weight']) * (df['mex_cy',1990,0]/df['mex_y',1990,0]) * (df['mex_y',2000,0] - df['mex_y',1990,0])

df.drop(['mex_cy','mex_y'], axis=1, inplace=True)

In [19]:
df.head()

col_share_cy                               fborn_share_cy  \
year                    1990.0              2000.0                   1990.0   
is_low                     0.0       1.0       0.0       1.0            0.0   
conspuma statefip                                                             
1.0      1.0          0.147911  0.147911  0.173970  0.173970       0.022487   
2.0      1.0          0.198305  0.198305  0.239633  0.239633       0.025164   
3.0      1.0          0.133413  0.133413  0.147069  0.147069       0.024208   
4.0      1.0          0.203165  0.203165  0.253293  0.253293       0.016145   
5.0      1.0          0.143687  0.143687  0.173467  0.173467       0.016575   

                                                female_share_cy            \
year                           2000.0                    1990.0             
is_low                  1.0       0.0       1.0             0.0       1.0   
conspuma statefip                                                           
1.0      1.0       0.022487  0.037724  0.037724        0.457208  0.457208   
2.0      1.0       0.025164  0.032525  0.032525        0.449781  0.449781   
3.0      1.0       0.024208  0.035858  0.035858        0.432028  0.432028   
4.0      1.0       0.016145  0.036248  0.036248        0.477015  0.477015   
5.0      1.0       0.016575  0.031662  0.031662        0.443216  0.443216   

                   ... mex_share_cy                                \
year               ...       1990.0              2000.0             
is_low             ...          0.0       1.0       0.0       1.0   
conspuma statefip  ...                                              
1.0      1.0       ...     0.001045  0.001045  0.001412  0.001412   
2.0      1.0       ...     0.000000  0.000000  0.002885  0.002885   
3.0      1.0       ...     0.000357  0.000357  0.008464  0.008464   
4.0      1.0       ...     0.000058  0.000058  0.008280  0.008280   
5.0      1.0       ...     0.000355  0.000355  0.007802  0.007802   

                  Dnilf_rate_c Dunemp_rate_c Dln_wage_c     weight         x  \
year                                                                           
is_low                                                                         
conspuma statefip                                                              
1.0      1.0         -0.007087      0.005717  -0.148429   223817.0 -0.047200   
2.0      1.0         -0.004367     -0.040523   0.124461    91314.0 -0.040416   
3.0      1.0          0.026795      0.028713   0.164947    70053.0 -0.044218   
4.0      1.0          0.025319      0.000125   0.092585   396086.0 -0.047242   
5.0      1.0         -0.021649     -0.005628  -0.026778  1649181.0 -0.065532   

                          z  
year                         
is_low                       
conspuma statefip            
1.0      1.0       0.001247  
2.0      1.0       0.000000  
3.0      1.0       0.000426  
4.0      1.0       0.000069  
5.0      1.0       0.000424  

[5 rows x 38 columns]

### Now run regressions! Show the first stage for 'x', then second stage regressions for NILF, wage, and unemployment rates